##Importação das Bibliotecas e conexao com o MONGO DB

Conectando ao Mongo DB

In [ ]:
#pip install pymongo[srv]
import pymongo
from pymongo import MongoClient

In [ ]:
#conector do Mongo Atlas
uri = "mongodb+srv://suzana-soulcode.a16wlwu.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&retryWrites=true&w=majority"
client = MongoClient(uri,tls=True,tlsCertificateKeyFile='/content/X509-cert-2478901955717142042.pem')


Importação das Bibiotecas

In [ ]:
# Importando Bibliotecas
import pandas as pd
import numpy as np

# Setando a exibição de no máximo 100 colunas
pd.set_option('display.max_columns',100)

#Carregamento dos Arquivos

## Integração com Pandas
Trazer o DataFrame do mongo : sample_supplies.sales

In [ ]:
db = client['sample_supplies']
supplies_sales = db['sales']
supplies_sales.count_documents({})

5000

In [ ]:
cursor = supplies_sales.find({}) #extracao via mongo db
df_sales = pd.DataFrame(list(cursor))

Verificando o DF

In [ ]:
df_sales

,_id,saleDate,items,storeLocation,customer,couponUsed,purchaseMethod
0,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506,"[{'name': 'printer paper', 'tags': ['office', ...",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@wit...",True,Online
1,5bd761dcae323e45a93ccfe9,2015-08-25 10:01:02.918,"[{'name': 'envelopes', 'tags': ['stationary', ...",Seattle,"{'gender': 'M', 'age': 50, 'email': 'keecade@h...",False,Phone
2,5bd761dcae323e45a93ccfea,2017-06-22 09:54:14.185,"[{'name': 'notepad', 'tags': ['office', 'writi...",Denver,"{'gender': 'M', 'age': 51, 'email': 'worbiduh@...",False,In store
3,5bd761dcae323e45a93ccfeb,2015-02-23 09:53:59.343,"[{'name': 'binder', 'tags': ['school', 'genera...",Seattle,"{'gender': 'F', 'age': 45, 'email': 'vatires@t...",False,In store
4,5bd761dcae323e45a93ccfec,2017-12-03 18:39:48.253,"[{'name': 'backpack', 'tags': ['school', 'trav...",London,"{'gender': 'M', 'age': 40, 'email': 'dotzu@ib....",False,In store
...,...,...,...,...,...,...,...
4995,5bd761deae323e45a93ce36b,2016-01-20 17:29:10.225,"[{'name': 'pens', 'tags': ['writing', 'office'...",Austin,"{'gender': 'F', 'age': 45, 'email': 'du@rum.py...",False,In store
4996,5bd761deae323e45a93ce36c,2014-11-29 10:09:41.573,"[{'name': 'pens', 'tags': ['writing', 'office'...",Denver,"{'gender': 'F', 'age': 47, 'email': 'dup@katje...",False,In store
4997,5bd761deae323e45a93ce36d,2017-12-25 07:02:51.283,"[{'name': 'binder', 'tags': ['school', 'genera...",Denver,"{'gender': 'M', 'age': 29, 'email': 'losuhse@e...",False,In store
4998,5bd761deae323e45a93ce36e,2013-12-07 11:06:25.510,"[{'name': 'notepad', 'tags': ['office', 'writi...",Denver,"{'gender': 'M', 'age': 59, 'email': 'hu@pehabu...",False,Online


In [ ]:
df_sales.shape

(5000, 7)

In [ ]:
df_sales.dtypes

_id                       object
saleDate          datetime64[ns]
items                     object
storeLocation             object
customer                  object
couponUsed                  bool
purchaseMethod            object
dtype: object

In [ ]:
df_sales.isna().sum()

_id               0
saleDate          0
items             0
storeLocation     0
customer          0
couponUsed        0
purchaseMethod    0
dtype: int64

# TRANSFORMAÇÕES
Verificado que existem dicionários a cada linha que precisam ser transformados em colunas
Tanto em itens quanto em customer

## Transformar dados colunas em novo dataframe (customer)

A coluna customer tem uma estrutura de dicionários em cada linha da tabela.
A intensão é tranformar em um novo dataframe e fazer um merge com o dataframe principal (df_sales)

In [ ]:
#Verificando dados do dicionário
df_sales.loc[0, 'customer']

{'gender': 'M', 'age': 42, 'email': 'cauho@witwuta.sv', 'satisfaction': 4}

In [ ]:
customer = df_sales['customer']
customer

0       {'gender': 'M', 'age': 42, 'email': 'cauho@wit...
1       {'gender': 'M', 'age': 50, 'email': 'keecade@h...
2       {'gender': 'M', 'age': 51, 'email': 'worbiduh@...
3       {'gender': 'F', 'age': 45, 'email': 'vatires@t...
4       {'gender': 'M', 'age': 40, 'email': 'dotzu@ib....
                              ...                        
4995    {'gender': 'F', 'age': 45, 'email': 'du@rum.py...
4996    {'gender': 'F', 'age': 47, 'email': 'dup@katje...
4997    {'gender': 'M', 'age': 29, 'email': 'losuhse@e...
4998    {'gender': 'M', 'age': 59, 'email': 'hu@pehabu...
4999    {'gender': 'M', 'age': 33, 'email': 'lihzodud@...
Name: customer, Length: 5000, dtype: object

In [ ]:
#Identificando as Keys para os 'Títulos' das futuras colunas relacionadas ao difionário
customer_dic = customer.loc[0]
customer_dic.keys()

dict_keys(['gender', 'age', 'email', 'satisfaction'])

In [ ]:
#Transformando as keys do dicionário em títulos das colunas que serão criadas
col_names = customer_dic.keys()
df_customer = pd.DataFrame(columns=col_names)

In [ ]:
#Separando as informações do dicionário em formato de tabela
for i in range(5000):
  df_customer = df_customer.append(customer.loc[i], ignore_index=True)

df_customer

,gender,age,email,satisfaction
0,M,42,cauho@witwuta.sv,4
1,M,50,keecade@hem.uy,5
2,M,51,worbiduh@vowbu.cg,5
3,F,45,vatires@ta.pe,3
4,M,40,dotzu@ib.sh,4
...,...,...,...,...
4995,F,45,du@rum.py,3
4996,F,47,dup@katjewtep.hr,3
4997,M,29,losuhse@eronehjod.np,4
4998,M,59,hu@pehabur.gd,4


Opções:
- Fazer download do arquivo e fazer o merge por index
- Fazer o reset no index e depois fazer o merge....e depois apagar todos esse indices colocados a mais

In [ ]:
#Usando o Reset_index no DF customer - Redefina o índice do DataFrame e use o padrão (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html)
df_customer.reset_index(inplace=True)
df_customer

,index,gender,age,email,satisfaction
0,0,M,42,cauho@witwuta.sv,4
1,1,M,50,keecade@hem.uy,5
2,2,M,51,worbiduh@vowbu.cg,5
3,3,F,45,vatires@ta.pe,3
4,4,M,40,dotzu@ib.sh,4
...,...,...,...,...,...
4995,4995,F,45,du@rum.py,3
4996,4996,F,47,dup@katjewtep.hr,3
4997,4997,M,29,losuhse@eronehjod.np,4
4998,4998,M,59,hu@pehabur.gd,4


In [ ]:
#Usando o Reset_index no DF principal - sales
df_sales.reset_index(inplace=True)
df_sales 

In [ ]:
df_sales

,index,_id,saleDate,items,storeLocation,customer,couponUsed,purchaseMethod
0,0,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506,"[{'name': 'printer paper', 'tags': ['office', ...",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@wit...",True,Online
1,1,5bd761dcae323e45a93ccfe9,2015-08-25 10:01:02.918,"[{'name': 'envelopes', 'tags': ['stationary', ...",Seattle,"{'gender': 'M', 'age': 50, 'email': 'keecade@h...",False,Phone
2,2,5bd761dcae323e45a93ccfea,2017-06-22 09:54:14.185,"[{'name': 'notepad', 'tags': ['office', 'writi...",Denver,"{'gender': 'M', 'age': 51, 'email': 'worbiduh@...",False,In store
3,3,5bd761dcae323e45a93ccfeb,2015-02-23 09:53:59.343,"[{'name': 'binder', 'tags': ['school', 'genera...",Seattle,"{'gender': 'F', 'age': 45, 'email': 'vatires@t...",False,In store
4,4,5bd761dcae323e45a93ccfec,2017-12-03 18:39:48.253,"[{'name': 'backpack', 'tags': ['school', 'trav...",London,"{'gender': 'M', 'age': 40, 'email': 'dotzu@ib....",False,In store
...,...,...,...,...,...,...,...,...
4995,4995,5bd761deae323e45a93ce36b,2016-01-20 17:29:10.225,"[{'name': 'pens', 'tags': ['writing', 'office'...",Austin,"{'gender': 'F', 'age': 45, 'email': 'du@rum.py...",False,In store
4996,4996,5bd761deae323e45a93ce36c,2014-11-29 10:09:41.573,"[{'name': 'pens', 'tags': ['writing', 'office'...",Denver,"{'gender': 'F', 'age': 47, 'email': 'dup@katje...",False,In store
4997,4997,5bd761deae323e45a93ce36d,2017-12-25 07:02:51.283,"[{'name': 'binder', 'tags': ['school', 'genera...",Denver,"{'gender': 'M', 'age': 29, 'email': 'losuhse@e...",False,In store
4998,4998,5bd761deae323e45a93ce36e,2013-12-07 11:06:25.510,"[{'name': 'notepad', 'tags': ['office', 'writi...",Denver,"{'gender': 'M', 'age': 59, 'email': 'hu@pehabu...",False,Online


In [ ]:
# Criando o merger entre o df principal e o de customer
dft = df_sales.merge(df_customer, how='outer')
dft.head(5)

,index,_id,saleDate,items,storeLocation,customer,couponUsed,purchaseMethod,gender,age,email,satisfaction
0,0,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506,"[{'name': 'printer paper', 'tags': ['office', ...",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@wit...",True,Online,M,42,cauho@witwuta.sv,4
1,1,5bd761dcae323e45a93ccfe9,2015-08-25 10:01:02.918,"[{'name': 'envelopes', 'tags': ['stationary', ...",Seattle,"{'gender': 'M', 'age': 50, 'email': 'keecade@h...",False,Phone,M,50,keecade@hem.uy,5
2,2,5bd761dcae323e45a93ccfea,2017-06-22 09:54:14.185,"[{'name': 'notepad', 'tags': ['office', 'writi...",Denver,"{'gender': 'M', 'age': 51, 'email': 'worbiduh@...",False,In store,M,51,worbiduh@vowbu.cg,5
3,3,5bd761dcae323e45a93ccfeb,2015-02-23 09:53:59.343,"[{'name': 'binder', 'tags': ['school', 'genera...",Seattle,"{'gender': 'F', 'age': 45, 'email': 'vatires@t...",False,In store,F,45,vatires@ta.pe,3
4,4,5bd761dcae323e45a93ccfec,2017-12-03 18:39:48.253,"[{'name': 'backpack', 'tags': ['school', 'trav...",London,"{'gender': 'M', 'age': 40, 'email': 'dotzu@ib....",False,In store,M,40,dotzu@ib.sh,4


## Transformar dados colunas em novo dataframe (items)

A coluna items tem uma estrutura de dicionários em cada linha da tabela.
A intensão é tranformar em um novo dataframe e fazer um merge com o novo dataframe principal (dft)

In [ ]:
#Verificando dados do dicionário
lista =dfsales.loc[0,'items']

In [ ]:
df = pd.DataFrame(list(lista))

In [ ]:
df

,name,tags,price,quantity
0,printer paper,"[office, stationary]",40.01,2
1,notepad,"[office, writing, school]",35.29,2
2,pens,"[writing, office, school, stationary]",56.12,5
3,backpack,"[school, travel, kids]",77.71,2
4,notepad,"[office, writing, school]",18.47,2
5,envelopes,"[stationary, office, general]",19.95,8
6,envelopes,"[stationary, office, general]",8.08,3
7,binder,"[school, general, organization]",14.16,3


In [ ]:
#Verificando dados do dicionário
df_sales.loc[0, 'items']

[{'name': 'printer paper',
  'tags': ['office', 'stationary'],
  'price': Decimal128('40.01'),
  'quantity': 2},
 {'name': 'notepad',
  'tags': ['office', 'writing', 'school'],
  'price': Decimal128('35.29'),
  'quantity': 2},
 {'name': 'pens',
  'tags': ['writing', 'office', 'school', 'stationary'],
  'price': Decimal128('56.12'),
  'quantity': 5},
 {'name': 'backpack',
  'tags': ['school', 'travel', 'kids'],
  'price': Decimal128('77.71'),
  'quantity': 2},
 {'name': 'notepad',
  'tags': ['office', 'writing', 'school'],
  'price': Decimal128('18.47'),
  'quantity': 2},
 {'name': 'envelopes',
  'tags': ['stationary', 'office', 'general'],
  'price': Decimal128('19.95'),
  'quantity': 8},
 {'name': 'envelopes',
  'tags': ['stationary', 'office', 'general'],
  'price': Decimal128('8.08'),
  'quantity': 3},
 {'name': 'binder',
  'tags': ['school', 'general', 'organization'],
  'price': Decimal128('14.16'),
  'quantity': 3}]

In [ ]:
items = df_sales['items']
items

0       [{'name': 'printer paper', 'tags': ['office', ...
1       [{'name': 'envelopes', 'tags': ['stationary', ...
2       [{'name': 'notepad', 'tags': ['office', 'writi...
3       [{'name': 'binder', 'tags': ['school', 'genera...
4       [{'name': 'backpack', 'tags': ['school', 'trav...
                              ...                        
4995    [{'name': 'pens', 'tags': ['writing', 'office'...
4996    [{'name': 'pens', 'tags': ['writing', 'office'...
4997    [{'name': 'binder', 'tags': ['school', 'genera...
4998    [{'name': 'notepad', 'tags': ['office', 'writi...
4999    [{'name': 'envelopes', 'tags': ['stationary', ...
Name: items, Length: 5000, dtype: object

In [ ]:
col_items = ['_id','name', 'tags', 'price', 'quantity']
#Transformando as keys do dicionário em títulos das colunas que serão criadas
df_items = pd.DataFrame(columns=col_items)
df_items

,_id,name,tags,price,quantity


In [ ]:
#Separando as informações do dicionário em formato de tabela
for i in range(5000):
  lista = df_sales.loc[i,'items']
  id = df_sales.loc[i,'_id']
  df = pd.DataFrame(list(lista))
  for j in range(len(df)):
    df_items = df_items.append({'_id' : id , 
                                'name' : df.loc[j,'name'], 
                                'price' : df.loc[j,'price'], 
                                'quantity':df.loc[j,'quantity'],
                                'tags':df.loc[j,'tags']} , 
                                ignore_index=True)

In [ ]:
df_items

,_id,name,tags,price,quantity
0,5bd761dcae323e45a93ccfe8,printer paper,"[office, stationary]",40.01,2
1,5bd761dcae323e45a93ccfe8,notepad,"[office, writing, school]",35.29,2
2,5bd761dcae323e45a93ccfe8,pens,"[writing, office, school, stationary]",56.12,5
3,5bd761dcae323e45a93ccfe8,backpack,"[school, travel, kids]",77.71,2
4,5bd761dcae323e45a93ccfe8,notepad,"[office, writing, school]",18.47,2
...,...,...,...,...,...
27433,5bd761deae323e45a93ce36f,envelopes,"[stationary, office, general]",24.12,5
27434,5bd761deae323e45a93ce36f,pens,"[writing, office, school, stationary]",27.1,4
27435,5bd761deae323e45a93ce36f,binder,"[school, general, organization]",29.09,3
27436,5bd761deae323e45a93ce36f,laptop,"[electronics, school, office]",598.79,2


Fazer o merger dessa vez pela coluna '_id'

In [ ]:
dft

,index,_id,saleDate,items,storeLocation,customer,couponUsed,purchaseMethod,gender,age,email,satisfaction
0,0,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506,"[{'name': 'printer paper', 'tags': ['office', ...",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@wit...",True,Online,M,42,cauho@witwuta.sv,4
1,1,5bd761dcae323e45a93ccfe9,2015-08-25 10:01:02.918,"[{'name': 'envelopes', 'tags': ['stationary', ...",Seattle,"{'gender': 'M', 'age': 50, 'email': 'keecade@h...",False,Phone,M,50,keecade@hem.uy,5
2,2,5bd761dcae323e45a93ccfea,2017-06-22 09:54:14.185,"[{'name': 'notepad', 'tags': ['office', 'writi...",Denver,"{'gender': 'M', 'age': 51, 'email': 'worbiduh@...",False,In store,M,51,worbiduh@vowbu.cg,5
3,3,5bd761dcae323e45a93ccfeb,2015-02-23 09:53:59.343,"[{'name': 'binder', 'tags': ['school', 'genera...",Seattle,"{'gender': 'F', 'age': 45, 'email': 'vatires@t...",False,In store,F,45,vatires@ta.pe,3
4,4,5bd761dcae323e45a93ccfec,2017-12-03 18:39:48.253,"[{'name': 'backpack', 'tags': ['school', 'trav...",London,"{'gender': 'M', 'age': 40, 'email': 'dotzu@ib....",False,In store,M,40,dotzu@ib.sh,4
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4995,5bd761deae323e45a93ce36b,2016-01-20 17:29:10.225,"[{'name': 'pens', 'tags': ['writing', 'office'...",Austin,"{'gender': 'F', 'age': 45, 'email': 'du@rum.py...",False,In store,F,45,du@rum.py,3
4996,4996,5bd761deae323e45a93ce36c,2014-11-29 10:09:41.573,"[{'name': 'pens', 'tags': ['writing', 'office'...",Denver,"{'gender': 'F', 'age': 47, 'email': 'dup@katje...",False,In store,F,47,dup@katjewtep.hr,3
4997,4997,5bd761deae323e45a93ce36d,2017-12-25 07:02:51.283,"[{'name': 'binder', 'tags': ['school', 'genera...",Denver,"{'gender': 'M', 'age': 29, 'email': 'losuhse@e...",False,In store,M,29,losuhse@eronehjod.np,4
4998,4998,5bd761deae323e45a93ce36e,2013-12-07 11:06:25.510,"[{'name': 'notepad', 'tags': ['office', 'writi...",Denver,"{'gender': 'M', 'age': 59, 'email': 'hu@pehabu...",False,Online,M,59,hu@pehabur.gd,4


In [ ]:
# Criando o merger entre o dft (novo principal) e o de items
dft2 = dft.merge(df_items, how='outer')


In [ ]:
dft2.head(5)

,index,_id,saleDate,items,storeLocation,customer,couponUsed,purchaseMethod,gender,age,email,satisfaction,name,tags,price,quantity
0,0,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506,"[{'name': 'printer paper', 'tags': ['office', ...",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@wit...",True,Online,M,42,cauho@witwuta.sv,4,printer paper,"[office, stationary]",40.01,2
1,0,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506,"[{'name': 'printer paper', 'tags': ['office', ...",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@wit...",True,Online,M,42,cauho@witwuta.sv,4,notepad,"[office, writing, school]",35.29,2
2,0,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506,"[{'name': 'printer paper', 'tags': ['office', ...",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@wit...",True,Online,M,42,cauho@witwuta.sv,4,pens,"[writing, office, school, stationary]",56.12,5
3,0,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506,"[{'name': 'printer paper', 'tags': ['office', ...",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@wit...",True,Online,M,42,cauho@witwuta.sv,4,backpack,"[school, travel, kids]",77.71,2
4,0,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506,"[{'name': 'printer paper', 'tags': ['office', ...",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@wit...",True,Online,M,42,cauho@witwuta.sv,4,notepad,"[office, writing, school]",18.47,2


## Drop colunas com dicionário

In [ ]:
#Drop das colunas no Novo DF principal
dft2.drop(['customer','items'], axis=1,inplace=True)

In [ ]:
#Drop da coluna index
dft2.drop(['index'], axis=1,inplace=True)

In [ ]:
#Novo DF Principal
dft2

,_id,saleDate,storeLocation,couponUsed,purchaseMethod,gender,age,email,satisfaction,name,tags,price,quantity
0,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506,Denver,True,Online,M,42,cauho@witwuta.sv,4,printer paper,"[office, stationary]",40.01,2
1,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506,Denver,True,Online,M,42,cauho@witwuta.sv,4,notepad,"[office, writing, school]",35.29,2
2,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506,Denver,True,Online,M,42,cauho@witwuta.sv,4,pens,"[writing, office, school, stationary]",56.12,5
3,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506,Denver,True,Online,M,42,cauho@witwuta.sv,4,backpack,"[school, travel, kids]",77.71,2
4,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506,Denver,True,Online,M,42,cauho@witwuta.sv,4,notepad,"[office, writing, school]",18.47,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27433,5bd761deae323e45a93ce36f,2014-08-18 06:25:49.739,New York,False,Online,M,33,lihzodud@voblep.ie,3,envelopes,"[stationary, office, general]",24.12,5
27434,5bd761deae323e45a93ce36f,2014-08-18 06:25:49.739,New York,False,Online,M,33,lihzodud@voblep.ie,3,pens,"[writing, office, school, stationary]",27.1,4
27435,5bd761deae323e45a93ce36f,2014-08-18 06:25:49.739,New York,False,Online,M,33,lihzodud@voblep.ie,3,binder,"[school, general, organization]",29.09,3
27436,5bd761deae323e45a93ce36f,2014-08-18 06:25:49.739,New York,False,Online,M,33,lihzodud@voblep.ie,3,laptop,"[electronics, school, office]",598.79,2


# Backup do novo DF 

In [ ]:
#backup do dataframe original
dfback = dft2.copy()

## Limpeza e Filtros no novo DF Principal

Transformar:

-'age' e 'quantity' em inteiro;

-'price' em float;


In [ ]:
dft2.dtypes

index                      int64
_id                       object
saleDate          datetime64[ns]
storeLocation             object
couponUsed                  bool
purchaseMethod            object
gender                    object
age                       object
email                     object
satisfaction              object
name                      object
tags                      object
price                     object
quantity                  object
dtype: object

In [ ]:
#consulta valores ausentes
dft2.isna().sum()

_id               0
saleDate          0
storeLocation     0
couponUsed        0
purchaseMethod    0
gender            0
age               0
email             0
satisfaction      0
name              0
tags              0
price             0
quantity          0
dtype: int64

In [ ]:
#verificar quais listas de elementos unicos naquela coluna
pd.unique(dft2['storeLocation'])

array(['Denver', 'Seattle', 'London', 'Austin', 'New York', 'San Diego'],
      dtype=object)

In [ ]:
pd.unique(dft2['gender'])

array(['M', 'F'], dtype=object)

## AGE 
Transformar para inteiro

In [ ]:
pd.unique(dft2['age'])

array([42, 50, 51, 45, 40, 44, 71, 26, 57, 34, 49, 59, 55, 36, 28, 53, 39,
       30, 31, 61, 58, 27, 48, 37, 70, 60, 54, 33, 66, 64, 52, 74, 29, 24,
       47, 68, 65, 38, 62, 25, 35, 46, 23, 63, 19, 21, 73, 18, 22, 69, 32,
       41, 17, 56, 43, 67, 72, 16, 75, 20], dtype=object)

In [ ]:
# trasformando object para inteiro
dft2['age'] = dft2['age'].astype(int)

In [ ]:
dft2.dtypes

_id                       object
saleDate          datetime64[ns]
storeLocation             object
couponUsed                  bool
purchaseMethod            object
gender                    object
age                        int64
email                     object
satisfaction              object
name                      object
tags                      object
price                     object
quantity                  object
dtype: object

## Quantity 
Transformar para inteiro

In [ ]:
pd.unique(dft2['quantity'])

array([2, 5, 8, 3, 10, 9, 4, 1, 7, 6], dtype=object)

In [ ]:
# trasformando object para inteiro
dft2['quantity'] = dft2['quantity'].astype(int)

In [ ]:
dft2.dtypes

_id                       object
saleDate          datetime64[ns]
storeLocation             object
couponUsed                  bool
purchaseMethod            object
gender                    object
age                        int64
email                     object
satisfaction              object
name                      object
tags                      object
price                     object
quantity                   int64
dtype: object

## PRICE
Transformar para float
Apesar de estar como object ele entende que é 'Decimal128'.
Tá dando erro pra corrigir o tipo

----------------------------
df['Field_2'] = df['Field_2'].apply(np.int64)
------------------------------
df['Coluna DataFrame'] = pd.to_numeric(df['Coluna DataFrame'])

In [ ]:
pd.unique(dft2['price'])

In [ ]:
# trasformando object para inteiro
dft2['price'] = dft2['price'].astype(float)

TypeError: ignored

In [ ]:
dft2.dtypes

In [ ]:
dft2


,_id,saleDate,storeLocation,couponUsed,purchaseMethod,gender,age,email,satisfaction,name,tags,price,quantity
0,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506,Denver,True,Online,M,42,cauho@witwuta.sv,4,printer paper,"[office, stationary]",40.01,2
1,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506,Denver,True,Online,M,42,cauho@witwuta.sv,4,notepad,"[office, writing, school]",35.29,2
2,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506,Denver,True,Online,M,42,cauho@witwuta.sv,4,pens,"[writing, office, school, stationary]",56.12,5
3,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506,Denver,True,Online,M,42,cauho@witwuta.sv,4,backpack,"[school, travel, kids]",77.71,2
4,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506,Denver,True,Online,M,42,cauho@witwuta.sv,4,notepad,"[office, writing, school]",18.47,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27433,5bd761deae323e45a93ce36f,2014-08-18 06:25:49.739,New York,False,Online,M,33,lihzodud@voblep.ie,3,envelopes,"[stationary, office, general]",24.12,5
27434,5bd761deae323e45a93ce36f,2014-08-18 06:25:49.739,New York,False,Online,M,33,lihzodud@voblep.ie,3,pens,"[writing, office, school, stationary]",27.1,4
27435,5bd761deae323e45a93ce36f,2014-08-18 06:25:49.739,New York,False,Online,M,33,lihzodud@voblep.ie,3,binder,"[school, general, organization]",29.09,3
27436,5bd761deae323e45a93ce36f,2014-08-18 06:25:49.739,New York,False,Online,M,33,lihzodud@voblep.ie,3,laptop,"[electronics, school, office]",598.79,2


##Load | Carregamento

In [ ]:
# utilizar quando uma for o rótulo padrão do pandas
#dft2.to_csv('xxxxxxx',index=False)